In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks/TP Vision/tp3+dataset"

/content/drive/My Drive/Colab Notebooks/TP Vision/tp3+dataset


In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from skimage import exposure
from skimage import feature
from imutils import paths
import imutils
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
	# Function to compute histogram from LBP features
	def compute_lbp(image, eps=1e-7,numPoints=24,radius=8):
		# compute the Local Binary Pattern representation
		# of the image, and then use the LBP representation
		# to build the histogram of patterns
		lbp = feature.local_binary_pattern(image, numPoints,
			radius, method="uniform")
		(hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, numPoints + 3),
			range=(0, numPoints + 2))
		# normalize the histogram
		hist = hist.astype("float")
		hist /= (hist.sum() + eps)
		# return the histogram of Local Binary Patterns
		return hist

# Feature extraction   from the training data with the corresponding labels

In [41]:
print("[INFO] extracting different features:  Raw Pixel values from image/Color Histogram/HOG/LBP")
fixed_width = 500
fixed_height = 364
train_data_hog=[]
#TODO : Add a vector to stock labels
labels = []
for imagePath in paths.list_images('caltech_dataset/train_set'):
  print (imagePath)
  #TODO: Retrieve the name of the class from each image path
  label = imagePath.split(os.path.sep)[-2]
  # Example of resizing images to a fixed size before HOG feature extraction
  image = cv2.imread(imagePath)
  image = cv2.resize(image, (fixed_width, fixed_height))  # Choose appropriate dimensions
  #TODO: Display the image
  #plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  #plt.title("Original Image")
  #plt.show()

  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  (H, hogImage) = feature.hog(gray_image, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2),transform_sqrt=True, block_norm="L1",visualize=True)
  hogImage = exposure.rescale_intensity(hogImage, out_range=(0, 255))
  hogImage = hogImage.astype("uint8")
  #TODO: Display HOG image
  #plt.imshow(hogImage, cmap='gray')
  #plt.title("HOG Image")
  #plt.show()

  train_data_hog.append(H)

  #TODO: update labels vector
  labels.append(label)

[INFO] extracting different features:  Raw Pixel values from image/Color Histogram/HOG/LBP
caltech_dataset/train_set/motorbike/14.png
caltech_dataset/train_set/motorbike/15.png
caltech_dataset/train_set/motorbike/18.png
caltech_dataset/train_set/motorbike/16.png
caltech_dataset/train_set/motorbike/19.png
caltech_dataset/train_set/motorbike/17.png
caltech_dataset/train_set/motorbike/20.png
caltech_dataset/train_set/motorbike/01.png
caltech_dataset/train_set/motorbike/02.png
caltech_dataset/train_set/motorbike/03.png
caltech_dataset/train_set/motorbike/04.png
caltech_dataset/train_set/motorbike/05.png
caltech_dataset/train_set/motorbike/06.png
caltech_dataset/train_set/motorbike/07.png
caltech_dataset/train_set/motorbike/08.png
caltech_dataset/train_set/motorbike/09.png
caltech_dataset/train_set/motorbike/10.png
caltech_dataset/train_set/motorbike/11.png
caltech_dataset/train_set/motorbike/12.png
caltech_dataset/train_set/motorbike/13.png
caltech_dataset/train_set/person/01.png
caltech_d

In [ ]:
#TODO: Print the size of each feature vector
print(f"Feature vector size for {imagePath}: {H.size}")

Feature vector size for caltech_dataset/train_set/car/12.png: 96624


# SVM and KNN models

In [ ]:
model1 = KNeighborsClassifier(n_neighbors=1)
model2 = SVC(kernel='linear')

# Feature extraction from the testing data with the corresponding labels

In [ ]:
test_data_hog = []
test_labels = []  # Initialize a list to store the test labels

# Assuming you have the testing set path 'caltech_dataset/test_set'
for imagePath in paths.list_images('caltech_dataset/test_set'):
    # Load the image and convert it to grayscale
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (fixed_width, fixed_height))  # Choose appropriate dimensions
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute HOG features (just as an example, add other features similarly)
    H = feature.hog(gray_image, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1")

    # Append the features to the test data list
    test_data_hog.append(H)

    # Retrieve the class label from the image path and append it to the test labels list
    label = imagePath.split(os.path.sep)[-2]
    test_labels.append(label)

    # Other feature computations can go here

# After the loop ends, you might need to convert test_data_hog and test_labels to numpy arrays if needed
# Convert lists to numpy arrays and ensure they are 2D (n_samples, n_features)
train_data_hog = np.array(train_data_hog)
test_data_hog = np.array(test_data_hog)

# Check if we need to reshape
if train_data_hog.ndim == 1:
    train_data_hog = train_data_hog.reshape(-1, 1)
if test_data_hog.ndim == 1:
    test_data_hog = test_data_hog.reshape(-1, 1)


# Train SVM and KNN models and predict the results, evaluate them as well!

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Assuming `train_data` and `train_labels` have been defined and preprocessed
model1.fit(train_data_hog, labels)
predicted_classes1 = model1.predict(test_data_hog)

# Evaluate the results by comparing the predicted classes to the ground truth (test_labels)
accuracy = accuracy_score(test_labels, predicted_classes1)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Additionally, you can get a full classification report that includes precision, recall, and F1-score for each class
class_report = classification_report(test_labels, predicted_classes1)
print("Classification Report:")
print(class_report)


Accuracy: 86.67%
Classification Report:
              precision    recall  f1-score   support

         car       0.83      1.00      0.91         5
   motorbike       1.00      0.80      0.89         5
      person       0.80      0.80      0.80         5

    accuracy                           0.87        15
   macro avg       0.88      0.87      0.87        15
weighted avg       0.88      0.87      0.87        15



In [ ]:
# Train and predict based on model 2, evalute your results as well!
from sklearn.metrics import classification_report, accuracy_score

# Assuming `train_data` and `train_labels` have been defined and preprocessed
model2.fit(train_data_hog, labels)
predicted_classes1 = model2.predict(test_data_hog)

# Evaluate the results by comparing the predicted classes to the ground truth (test_labels)
accuracy = accuracy_score(test_labels, predicted_classes1)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Additionally, you can get a full classification report that includes precision, recall, and F1-score for each class
class_report = classification_report(test_labels, predicted_classes1)
print("Classification Report:")
print(class_report)


Accuracy: 93.33%
Classification Report:
              precision    recall  f1-score   support

         car       0.83      1.00      0.91         5
   motorbike       1.00      1.00      1.00         5
      person       1.00      0.80      0.89         5

    accuracy                           0.93        15
   macro avg       0.94      0.93      0.93        15
weighted avg       0.94      0.93      0.93        15



# Feature Selection by PCA

In [ ]:
#TODO: Apply PCA  on LBP features, retrain the two models after selection  and predict the results, dont forget to evaluate and to compare regarding the original LBP features
scaler = StandardScaler()
train_data_hog_standardized  = scaler.fit_transform(train_data_hog)
test_data_hog_standardized = scaler.transform(test_data_hog)

# Step 3: Apply PCA
pca = PCA(n_components=0.28, whiten=True)
train_data_hog_pca = pca.fit_transform(train_data_hog_standardized)
test_data_hog_pca = pca.transform(test_data_hog_standardized)

# Retrain models with the PCA-transformed features
model1.fit(train_data_hog_pca, labels)
model2.fit(train_data_hog_pca, labels)

# Step 5: Predict and evaluate the results
predicted_classes1 = model1.predict(test_data_hog_pca)
predicted_classes2 = model2.predict(test_data_hog_pca)

accuracy1 = accuracy_score(test_labels, predicted_classes1)
accuracy2 = accuracy_score(test_labels, predicted_classes2)

print(f"Model 1 Accuracy with PCA: {accuracy1 * 100:.2f}%")
print(f"Model 2 Accuracy with PCA: {accuracy2 * 100:.2f}%")

class_report1 = classification_report(test_labels, predicted_classes1)
class_report2 = classification_report(test_labels, predicted_classes2)

print("Model 1 Classification Report with PCA:")
print(class_report1)

print("Model 2 Classification Report with PCA:")
print(class_report2)


Model 1 Accuracy with PCA: 80.00%
Model 2 Accuracy with PCA: 80.00%
Model 1 Classification Report with PCA:
              precision    recall  f1-score   support

         car       0.71      1.00      0.83         5
   motorbike       1.00      0.40      0.57         5
      person       0.83      1.00      0.91         5

    accuracy                           0.80        15
   macro avg       0.85      0.80      0.77        15
weighted avg       0.85      0.80      0.77        15

Model 2 Classification Report with PCA:
              precision    recall  f1-score   support

         car       0.83      1.00      0.91         5
   motorbike       1.00      0.60      0.75         5
      person       0.67      0.80      0.73         5

    accuracy                           0.80        15
   macro avg       0.83      0.80      0.80        15
weighted avg       0.83      0.80      0.80        15

